In [1]:
!pip install astropy --upgrade
!pip install photutils

import numpy as np
import math
import matplotlib.pyplot as plt
import os
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
import glob
import warnings
warnings.filterwarnings("ignore")
from matplotlib.gridspec import GridSpec

   ---------------------------------------- 0.0/6.5 MB ? eta -:--:--
   ----------- ---------------------------- 1.8/6.5 MB 9.1 MB/s eta 0:00:01
   --------------------------- ------------ 4.5/6.5 MB 10.7 MB/s eta 0:00:01
   -------------------------------------- - 6.3/6.5 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------- 6.5/6.5 MB 10.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   -------------------------- ------------- 1.3/1.9 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 5.1 MB/s eta 0:00:00
  Attempting uninstall: astropy-iers-data
    Found existing installation: astropy-iers-data 0.2024.4.15.2.45.49
    Uninstalling astropy-iers-data-0.2024.4.15.2.45.49:
      Successfully uninstalled astropy-iers-data-0.2024.4.15.2.45.49
  Attempting uninstall: astropy
    Found existing installation: astropy 7.0.0
    Uninstalling astropy-7.0.0:
      Successfully uninstalled astropy-7.0.0



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/756.2 kB ? eta -:--:--
   ---------------------------------------- 756.2/756.2 kB 6.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
